# 자식 프로세스를 관리하려면 subprocess를 사용하자

Python은 실전에서 단련된 자식 프로세스 실행과 관리용 라이브러리를 갖추고 있다. 따라서 명령줄 유틸리티 같은 다른 도구들을 연계하는 데 아주 좋은 언어다.

파이썬으로 시작한 자식 프로세스는 병렬로 실행할 수 있으므로, 파이썬을 사용하면 머신의 CPU 코어를 모두 이용해 프로그램의 처리량을 극대화 할 수 있다. 파이썬 자체는 CPU 속도에 의존할 수 있지만, CPU를 많이 사용하는 작업을 관리하고 조절하기 쉽다.

파이썬의 여러가지 서브프로세스 실행 방법: `popen`, `popen2`, `os.exec*`. 요즘의 파이썬에서 자식 프로세스를 관리하는 최선이자 가장 간단한 방법은 내장 모듈 subprocess를 사용하는 것이다.

subprocess로 자식 프로세스를 실행하는 방법이다. 다음 코드에서는 Popen 생성자가 프로세스를 시작한다. communicate 메서드는 자식 프로세스의 출력을 읽어오고 자식 프로세스가 종료할 때까지 기다린다.

In [51]:
import subprocess

proc = subprocess.Popen(['echo', 'Hello from the child!'], stdout=subprocess.PIPE)
out, err = proc.communicate()
print(out.decode('utf-8'))

Hello from the child!



자식 프로세스는 부모 프로세스와 파이썬 인터프리터와는 독립적으로 실행된다. 자식 프로세스의 상태는 파이썬이 다른 작업을 하는 동안 주기적으로 polling 된다.

In [52]:
proc = subprocess.Popen(['sleep', '0.3'])
while proc.poll() is None:
    print('Working...')
    # 시간이 걸리는 작업 몇 개를 수행함
    temp = False
    for i in range(16):
        for j in range(i):
            if i * j % 2 == 0:
                temp = !temp
print('Exit status', proc.poll())

Working...
Exit status 0


부모에서 자식 프로세스를 떼어낸다는 건 부모 프로세스가 자유롭게 여러 자식 프로세스를 병렬로 실행할 수 있음을 의미한다. 자식 프로세스를 떼어내려면 모든 자식 프로세스를 먼저 시작하면 된다.

In [53]:
from time import time

def run_sleep(period):
    proc = subprocess.Popen(['sleep', str(period)])
    return proc

start = time()
procs = []
for _ in range(10):
    proc = run_sleep(0.1)
    procs.append(proc)

이후 communicate 메서드로 자식 프로세스들이 I/O를 마치고 종료하기를 기다리면 된다.

In [54]:
for proc in procs:
    proc.communicate()
end = time()
print('Finished in %.3f seconds' % (end - start))

Finished in 1.469 seconds


Python 프로그램에서 Pipe를 이용해 데이터를 서브 프로세스로 보낸 다음 서브 프로세스의 결과를 받아올 수도 있다. 이 방법을 이용하면 다른 프로그램을 활용하여 작업을 병렬로 수행할 수 있다. 예를 들어 어떤 데이터를 암호화하는데 openssl 명령줄 도구를 사용하려 한다고 하자. 명령줄 인수와 I/O 파이프를 사용하여 자식 프로세스를 실행하는 건 간단하다.

In [55]:
import os

def run_openssl(data):
    env = os.environ.copy()
    env['password'] = b'\xe24U\n\xd0Ql3S\x11'
    proc = subprocess.Popen(['openssl', 'enc', '-des3', '-pass', 'env:password'],
                           env=env,
                           stdin=subprocess.PIPE,
                           stdout=subprocess.PIPE)
    proc.stdin.write(data)
    proc.stdin.flush() # 자식 프로세스가 입력을 반드시 받게 함
    return proc

예제에서는 파이프로 암호화 함수에 임의의 바이트를 전달하지만 실전에서는 사용자 입력, 파일 핸들, 네트워크 소켓 등을 전달할 것이다.

In [58]:
procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    procs.append(proc)

자식 프로세스는 병렬로 실행되고 입력을 소비한다. 다음 코드에서는 자식 프로세스가 종료할 때까지 대기하고 최종 결과를 받는다.

In [59]:
for proc in procs:
    out, err = proc.communicate()
    print(out[-10:])

b'\xea\xec\xc2\x16\xb9\xfa\x97+\xfe\x9b'
b'\xf6\xdc\x9es\x1c\xb3-+\x9d\x84'
b'CEX\\\xb6\xab\x8b\xee\rJ'


유닉스의 파이프처럼 한 자식 프로세스의 결과를 다른 프로세스의 입력으로 연결하여 병렬 프로세스의 chain을 생성할 수도 있다. 다음은 자식 프로세스를 시작하여 `md5` 명령줄 도구에서 입력 스트림을 소비하게 하는 함수다.

In [60]:
def run_md5(input_stdin):
    proc = subprocess.Popen(
    ['md5'],
    stdin=input_stdin,
    stdout=subprocess.PIPE)
    return proc

이제 데이터를 암호화하는 openssl 프로세스 집합과 암호화된 결과를 `md5`로 hash하는 프로세스 집합을 시작할 수 있다.

In [62]:
input_procs = []
hash_procs = []
for _ in range(3):
    data = os.urandom(10)
    proc = run_openssl(data)
    input_procs.append(proc)
    hash_proc = run_md5(proc.stdout)
    hash_procs.append(hash_proc)

일단 자식 프로세스들이 시작하면 이들 사이의 I/O는 자동으로 일어난다. 할 일은 모든 작업이 끝나고 최종 결과물이 출력되기를 기다리는 것 뿐이다.

In [63]:
for proc in input_procs:
    proc.communicate()
    
for proc in hash_procs:
    out, err = proc.communicate()
    print(out.strip())

b'40d193f2ff670c5ae8bb7223268073a3'
b'788ebb0208d812f8d6f7971823caa08e'
b'6d6007a9f4078c7336c2a05f5f41513c'


자식 프로세스가 종료되지 않거나 입력 또는 출력 파이프에서 블록될 염려가 있다면 communicate 메서드에 timeout 파라미터를 넘겨야 한다.

자식 프로세스가 일정한 시간 내에 응답하지 않을 때, 예외가 일어나서 오동작하는 자식 프로세스를 종료할 기회를 얻는다.

In [65]:
proc = run_sleep(10)
try:
    proc.communicate(timeout=0.1)
except subprocess.TimeoutExpired:
    proc.terminate()
    proc.wait()
    
print('Exit status', proc.poll())

Exit status -15


불행히도 timeout 파라미터는 Python3.3과 이후 버전에서 사용 가능하다. 이전 버전에서 I/O timeout을 강제하려면 내장 모듈 select를 `proc.stdin`, `proc.stdout`, `proc.stderr`에 사용해야 한다.